# Project: Data Cleaning Challenge

In [1]:
# Importing the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import unicodedata


# display outputs inline
%matplotlib inline

# to see all columns and rows to using the display max column parameter
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# set the display colwidth for seeing the entire text contents
pd.set_option('display.max_colwidth', -1)

C:\Users\USER\AppData\Local\Temp\ipykernel_11652\2121266886.py:17: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [2]:
# Importing data
fifa = pd.read_csv('fifa21 raw data v2.csv')

C:\Users\USER\AppData\Local\Temp\ipykernel_11652\3160915589.py:2: DtypeWarning: Columns (76) have mixed types. Specify dtype option on import or set low_memory=False.
  fifa = pd.read_csv('fifa21 raw data v2.csv')


In [3]:
# make a copy of the data so as not to tamper with the original copy
fifa_clean = fifa.copy()

# Data Inspection

In [4]:
# view a sample of the data
fifa_clean.sample(5)

,ID,Name,LongName,photoUrl,playerUrl,Nationality,Age,↓OVA,POT,Club,Contract,Positions,Height,Weight,Preferred Foot,BOV,Best Position,Joined,Loan Date End,Value,Wage,Release Clause,Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,W/F,SM,A/W,D/W,IR,PAC,SHO,PAS,DRI,DEF,PHY,Hits
14087,241197,A. Al Amri,Abdulelah Al Amri,https://cdn.sofifa.com/players/241/197/21_60.png,http://sofifa.com/player/241197/abdulelah-al-amri/210006/,Saudi Arabia,23,61,71,\n\n\n\nAl Nassr,2015 ~ 2021,CB,185cm,74kg,Right,63,CB,"Dec 3, 2015",NaN,€675K,€5K,€713K,191,33,20,63,54,21,179,33,26,32,48,40,273,53,56,55,57,52,251,40,68,49,72,22,220,47,61,29,38,45,45,188,62,64,62,53,11,8,13,9,12,1355,287,3 ★,2★,Medium,Medium,1 ★,55,26,43,39,63,61,2
2278,199559,R. Inglese,Roberto Inglese,https://cdn.sofifa.com/players/199/559/21_60.png,http://sofifa.com/player/199559/roberto-inglese/210006/,Italy,28,74,74,\n\n\n\nParma,2020 ~ 2024,ST,186cm,85kg,Right,74,ST,"Sep 1, 2020",NaN,€4.3M,€31K,€10.8M,342,39,78,90,65,70,280,68,68,36,35,73,272,43,58,50,70,51,361,78,79,57,79,68,251,40,19,78,42,72,71,55,30,13,12,55,14,11,10,13,7,1616,335,3 ★,3★,Medium,Medium,2 ★,51,75,49,67,27,66,14
2166,206407,João Pereira,João Pedro da Silva Pereira,https://cdn.sofifa.com/players/206/407/21_60.png,http://sofifa.com/player/206407/joao-pedro-da-silva-pereira/210006/,Portugal,36,74,74,\n\n\n\nTrabzonspor,2017 ~ 2021,RB,172cm,66kg,Right,74,RB,"Jan 4, 2017",NaN,€925K,€17K,€2M,315,75,58,57,73,52,349,75,72,60,69,73,364,68,72,73,71,80,335,68,75,75,56,61,359,86,72,69,68,64,71,218,72,74,72,53,12,14,10,10,7,1993,415,3 ★,3★,High,Medium,2 ★,70,61,71,74,71,68,14
17390,245485,P. Samiec-Talar,Piotr Samiec-Talar,https://cdn.sofifa.com/players/245/485/21_60.png,http://sofifa.com/player/245485/piotr-samiec-talar/210006/,Poland,18,56,74,\n\n\n\nŚląsk Wrocław,2018 ~ 2023,ST,185cm,78kg,Left,58,ST,"Jul 18, 2018",NaN,€375K,€700,€375K,237,31,59,54,45,48,210,55,33,31,34,57,335,73,69,69,57,67,274,52,65,54,61,42,238,58,25,53,45,57,43,76,31,21,24,59,14,8,14,13,10,1429,308,3 ★,2★,Medium,Medium,1 ★,71,53,39,58,28,59,NaN
2776,142966,J. Dátolo,Jesús Dátolo,https://cdn.sofifa.com/players/142/966/21_60.png,http://sofifa.com/player/142966/jesus-datolo/210006/,Argentina,36,73,73,\n\n\n\nClub Atlético Banfield,2017 ~ 2021,"CAM, CM",179cm,77kg,Left,73,CAM,"Jul 17, 2017",NaN,€1M,€10K,€2M,326,63,73,47,76,67,380,75,80,77,72,76,310,55,53,66,72,64,314,78,44,56,60,76,324,44,55,71,79,75,73,132,49,43,40,56,10,8,11,14,13,1842,378,3 ★,4★,High,Low,1 ★,54,74,74,74,47,55,8


#### Observations
1. Name of clubs are messy, they contain irrelevant characters
2. The Contract column values dont follow the same format
3. The Height column values arent well formatted so would likely give rise to wrong datatype, this is observed in the weight column too.
4. The weight column is graduated in the wrong units, it ought to be in lbs
5. Loan end date looks to contain lots of NAN values.
6. The Value, Wage and Release Clause columns arent well formatted, the currency type should be in dollars and some conversions should be made.
7. The W/F, SM and IR columns are not well formatted and would likely give rise to the wrong datatype
8. Some Column Names arent descriptive enough.
9. There are inconsistencies in some values, like in Hits column where some data have a K in them.

In [5]:
# Checking for duplicates
fifa_clean[fifa_clean.duplicated()]

,ID,Name,LongName,photoUrl,playerUrl,Nationality,Age,↓OVA,POT,Club,Contract,Positions,Height,Weight,Preferred Foot,BOV,Best Position,Joined,Loan Date End,Value,Wage,Release Clause,Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,W/F,SM,A/W,D/W,IR,PAC,SHO,PAS,DRI,DEF,PHY,Hits


#### No duplicate data in the dataset


In [6]:
# Identifying Null Values
null_counts = fifa_clean.isnull().sum()  # Count the number of null values in each column
null_cols = null_counts[null_counts > 0].index.tolist()  # Get the column names with null values

# Create a new dataframe with only the columns with null values and their corresponding null counts
null_fifa = pd.DataFrame({'Columns': null_cols, 'Null Count': null_counts[null_cols].tolist()})

null_fifa

,Columns,Null Count
0,Loan Date End,17966
1,Hits,2595


#### Observations
1. Only two columns contain Null Values, the Loan Date End and the Hits column, the Loan Date End Nulls seem to be significant.

In [7]:
# checking the datatypes and other information for each columns for each column
fifa_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18979 entries, 0 to 18978
Data columns (total 77 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ID                18979 non-null  int64 
 1   Name              18979 non-null  object
 2   LongName          18979 non-null  object
 3   photoUrl          18979 non-null  object
 4   playerUrl         18979 non-null  object
 5   Nationality       18979 non-null  object
 6   Age               18979 non-null  int64 
 7   ↓OVA              18979 non-null  int64 
 8   POT               18979 non-null  int64 
 9   Club              18979 non-null  object
 10  Contract          18979 non-null  object
 11  Positions         18979 non-null  object
 12  Height            18979 non-null  object
 13  Weight            18979 non-null  object
 14  Preferred Foot    18979 non-null  object
 15  BOV               18979 non-null  int64 
 16  Best Position     18979 non-null  object
 17  Joined      

#### Observation
1. There are some inappropriate data type assigned ro some columns

In [8]:
# view summary statistics of the fifa data
fifa_clean.describe()

,ID,Age,↓OVA,POT,BOV,Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,PAC,SHO,PAS,DRI,DEF,PHY
count,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.00000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000
mean,226403.384794,25.194109,65.718636,71.136414,66.751726,248.938142,49.688392,45.842405,51.942726,58.768112,42.696507,256.479214,55.587491,47.246957,42.391011,52.725381,58.528373,317.718689,64.357553,64.412930,63.366721,61.655619,63.925865,296.605195,57.801676,64.577375,62.653986,64.786922,46.785236,254.199747,55.591285,46.394489,50.298962,53.857105,48.057906,58.07366,139.909268,46.559197,47.705622,45.644449,81.436693,16.416302,16.207124,16.076453,16.217187,16.519627,1595.286949,355.702197,67.453975,53.457031,57.681016,62.875020,49.866221,64.368934
std,27141.054157,4.710520,6.968999,6.114635,6.747193,74.299428,18.131153,19.567081,17.294409,14.519106,17.646937,78.650601,18.761314,18.207790,17.227947,15.178151,16.565892,55.879046,14.889981,14.638743,14.596277,9.072114,14.062285,50.732390,13.308747,11.844870,15.804223,12.488672,19.300534,64.595613,17.140414,20.698078,19.428701,13.708574,15.655999,12.07911,61.219472,20.139324,21.367690,20.922087,84.665579,17.554194,16.816305,16.491103,17.002239,17.854079,269.874789,40.761117,10.677859,13.827425,10.081857,9.927415,16.443213,9.601883
min,41.000000,16.000000,47.000000,47.000000,48.000000,42.000000,6.000000,3.000000,5.000000,7.000000,3.000000,40.000000,5.000000,4.000000,5.000000,5.000000,5.000000,122.000000,13.000000,12.000000,14.000000,24.000000,12.000000,122.000000,18.000000,15.000000,12.000000,16.000000,4.000000,50.000000,9.000000,3.000000,2.000000,9.000000,6.000000,12.00000,20.000000,3.000000,5.000000,4.000000,10.000000,2.000000,2.000000,2.000000,2.000000,2.000000,747.000000,232.000000,25.000000,16.000000,25.000000,25.000000,12.000000,28.000000
25%,210135.000000,21.000000,61.000000,67.000000,62.000000,222.000000,38.000000,30.000000,44.000000,54.000000,30.000000,222.000000,49.000000,35.000000,31.000000,43.000000,54.000000,289.000000,57.000000,57.000000,55.000000,56.000000,56.000000,264.000000,48.000000,58.000000,55.000000,57.000000,32.000000,227.500000,44.000000,25.000000,40.000000,45.000000,39.000000,50.00000,83.000000,29.000000,27.000000,25.000000,48.000000,8.000000,8.000000,8.000000,8.000000,8.000000,1452.000000,327.000000,61.000000,44.000000,51.000000,57.000000,35.000000,58.000000
50%,232418.000000,25.000000,66.000000,71.000000,67.000000,263.000000,54.000000,49.000000,55.000000,62.000000,44.000000,269.000000,61.000000,49.000000,41.000000,56.000000,63.000000,327.000000,67.000000,67.000000,66.000000,62.000000,66.000000,302.000000,59.000000,65.000000,66.000000,66.000000,51.000000,263.000000,58.000000,53.000000,55.000000,55.000000,49.000000,59.00000,159.000000,52.000000,55.000000,52.000000,53.000000,11.000000,11.000000,11.000000,11.000000,11.000000,1627.000000,356.000000,68.000000,56.000000,58.000000,64.000000,53.000000,65.000000
75%,246922.500000,29.000000,70.000000,75.000000,71.000000,297.

# Data Cleaning

### Handling inconsistencies in the data values

#### Rename Columns

In [9]:
fifa_clean.columns

Index(['ID', 'Name', 'LongName', 'photoUrl', 'playerUrl', 'Nationality', 'Age',
       '↓OVA', 'POT', 'Club', 'Contract', 'Positions', 'Height', 'Weight',
       'Preferred Foot', 'BOV', 'Best Position', 'Joined', 'Loan Date End',
       'Value', 'Wage', 'Release Clause', 'Attacking', 'Crossing', 'Finishing',
       'Heading Accuracy', 'Short Passing', 'Volleys', 'Skill', 'Dribbling',
       'Curve', 'FK Accuracy', 'Long Passing', 'Ball Control', 'Movement',
       'Acceleration', 'Sprint Speed', 'Agility', 'Reactions', 'Balance',
       'Power', 'Shot Power', 'Jumping', 'Stamina', 'Strength', 'Long Shots',
       'Mentality', 'Aggression', 'Interceptions', 'Positioning', 'Vision',
       'Penalties', 'Composure', 'Defending', 'Marking', 'Standing Tackle',
       'Sliding Tackle', 'Goalkeeping', 'GK Diving', 'GK Handling',
       'GK Kicking', 'GK Positioning', 'GK Reflexes', 'Total Stats',
       'Base Stats', 'W/F', 'SM', 'A/W', 'D/W', 'IR', 'PAC', 'SHO', 'PAS',
       'DRI', 'DEF', 

In [10]:
# Give columns more descriptive names for easy understanding, Abbreviations can be ambiguous and can lead to misunderstandings when analyzing the data
fifa_clean.rename(columns = {'LongName':'Full Name','Age':'Age(2021)', '↓OVA':'Overall Rating', 'POT':'Potential Rating',
                             'Height':'Height(ft)', 'Weight':'Weight(lbs)', 'BOV':'Best Overall Rating',
                            'Value':'Value(€)', 'Wage':'Wage(€)', 'ReleaseClause':'Release Clause(€)',
                            'W/F':'Weaker Foot(★)', 'SM':'Skils(★)', 'A/W':'Attacking Contribution',
                             'D/W':'Defensive Contribution', 'IR':'Injury Resistance(★)', 'PAC':'Pace', 'SHO':'Shot', 
                             'PAS':'Pass Accuracy', 'DRI':'Dribbling Ability',
                            'DEF':'Defensive Ability', 'PHY': 'Physicality', 'Hits': 'Profile Views'}, inplace = True)

fifa_clean.columns

Index(['ID', 'Name', 'Full Name', 'photoUrl', 'playerUrl', 'Nationality',
       'Age(2021)', 'Overall Rating', 'Potential Rating', 'Club', 'Contract',
       'Positions', 'Height(ft)', 'Weight(lbs)', 'Preferred Foot',
       'Best Overall Rating', 'Best Position', 'Joined', 'Loan Date End',
       'Value(€)', 'Wage(€)', 'Release Clause', 'Attacking', 'Crossing',
       'Finishing', 'Heading Accuracy', 'Short Passing', 'Volleys', 'Skill',
       'Dribbling', 'Curve', 'FK Accuracy', 'Long Passing', 'Ball Control',
       'Movement', 'Acceleration', 'Sprint Speed', 'Agility', 'Reactions',
       'Balance', 'Power', 'Shot Power', 'Jumping', 'Stamina', 'Strength',
       'Long Shots', 'Mentality', 'Aggression', 'Interceptions', 'Positioning',
       'Vision', 'Penalties', 'Composure', 'Defending', 'Marking',
       'Standing Tackle', 'Sliding Tackle', 'Goalkeeping', 'GK Diving',
       'GK Handling', 'GK Kicking', 'GK Positioning', 'GK Reflexes',
       'Total Stats', 'Base Stats', 'Weaker

#### Datatype Conversion

In [11]:
fifa_clean['ID'] = fifa_clean['ID'].astype(str)
fifa_clean['Preferred Foot'] = fifa_clean['Preferred Foot'].astype('category')
fifa_clean['Joined'] = pd.to_datetime(fifa_clean['Joined'])
fifa_clean['Loan Date End'] = pd.to_datetime(fifa_clean['Loan Date End'])
fifa_clean['Weaker Foot(★)'] = fifa_clean['Weaker Foot(★)'].astype('category')
fifa_clean['Skils(★)'] = fifa_clean['Skils(★)'].astype('category')
fifa_clean['Attacking Contribution'] = fifa_clean['Attacking Contribution'].astype('category')
fifa_clean['Defensive Contribution'] = fifa_clean['Defensive Contribution'].astype('category')
fifa_clean['Injury Resistance(★)'] = fifa_clean['Injury Resistance(★)'].astype('category')

#### Cleaning the Profile Views Column

This column has 2595 missing  values, values of over 1000 are represented with a decimal and a k, handling these issue would make the column cleaner, To fill the missing values, the mode would be used as the data is skewed

In [12]:
# looking at the Profile Views column to observe inconsistencies
fifa_clean['Profile Views'].loc[6:8]

6    246 
7    120 
8    1.6K
Name: Profile Views, dtype: object

In [13]:
# Replace 'K' and 'M' with their corresponding numeric values in the 'Profile Views' column
fifa_clean['Profile Views'] = fifa_clean['Profile Views'].str.replace('K', 'e3').str.replace('M', 'e6')

# Convert the 'Profile Views' column to numeric type
fifa_clean['Profile Views'] = pd.to_numeric(fifa_clean['Profile Views'])

# Filter rows where the 'Profile Views' column contains 'K'
k_values = fifa_clean[fifa_clean['Profile Views'].astype(str).str.contains('e3')]

fifa_clean['Profile Views'].loc[6:8]

6    246.0 
7    120.0 
8    1600.0
Name: Profile Views, dtype: float64

In [14]:
# check the skewness of the hits values to determine what measure of center is best to fill null values
fifa_clean['Profile Views'].skew()

31.04253072771258

In [15]:
# get the mode to fill in na with
mode_hits = fifa_clean['Profile Views'].mode()
mode_hits

0    1.0
Name: Profile Views, dtype: float64

In [16]:
# fill in missing values in Profile Views column
fifa_clean['Profile Views'].fillna(1.0, inplace=True)

# change profile views to int data type
fifa_clean['Profile Views'] = fifa_clean['Profile Views'].astype(int)

# Check to confirm there are no Null values
fifa_clean[fifa_clean['Profile Views'].isna()]

,ID,Name,Full Name,photoUrl,playerUrl,Nationality,Age(2021),Overall Rating,Potential Rating,Club,Contract,Positions,Height(ft),Weight(lbs),Preferred Foot,Best Overall Rating,Best Position,Joined,Loan Date End,Value(€),Wage(€),Release Clause,Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,Weaker Foot(★),Skils(★),Attacking Contribution,Defensive Contribution,Injury Resistance(★),Pace,Shot,Pass Accuracy,Dribbling Ability,Defensive Ability,Physicality,Profile Views


####  Cleaning the Contract column

The Contract Column is having many discrepancies in value format, it also has some relationship with the Joined and loan date end column, from the contract column, a contract type column would be derived, this column tells us if the players contract is a permanent deal, a loan deal or a free agent(not under contract with any football club and is therefore free to sign with any club of their choosing). After which a contract start and contract end column is derived from the contract column, joined column and loan date column, these new columns tells when a player contract started, for a free agent this would be non applicable (N/A) as he has no contract while the contract end tells when the contracts would be terminated, for free agents the contract end would be N/A (not applicable) as a player without a contract cant have end date

In [17]:
fifa_clean['Contract'].loc[397:405]

397    Free                
398    2019 ~ 2023         
399    2017 ~ 2023         
400    2019 ~ 2024         
401    2014 ~ 2026         
402    2018 ~ 2023         
403    2019 ~ 2023         
404    2019 ~ 2024         
405    Jun 30, 2021 On Loan
Name: Contract, dtype: object

In [18]:
fifa_clean['Contract Type'] = fifa_clean['Contract'].apply(lambda x: 'Free' if x == 'Free' 
                                         else 'On Loan' if 'On Loan' in x 
                                         else 'Permanent' if '~' in x 
                                         else np.nan)


In [19]:
# define a function to parse the values in the 'Joined' and 'Loan Date End' columns
def parse_dates(row):
    if row['Contract'] == 'Free':
        start = np.nan
        end = np.nan
    elif 'On Loan' in row['Contract']:
        start = pd.to_datetime(row['Joined']).year
        end = row['Contract'].split(' ')[2]
    else:
        start = int(row['Contract'][:4])
        end = int(row['Contract'][-4:])
    return pd.Series({'Contract Start': start, 'Contract End': end})

# apply the function to create the new columns
fifa_clean[['Contract Start', 'Contract End']] = fifa_clean.apply(parse_dates, axis=1)


# convert float columns to object datatype and remove the unwanted characters
fifa_clean['Contract Start'] = fifa_clean['Contract Start'].astype(str)
fifa_clean['Contract End'] = fifa_clean['Contract End'].astype(str)

# Define a function to remove the '.i0' part from a string
def remove_dot_i(s):
    return s.split('.')[0]

# Apply the function to the fifa_clean['Contract Start']  and fifa_clean['Contract end'] column using .map()
fifa_clean['Contract Start'] = fifa_clean['Contract Start'].map(remove_dot_i)
fifa_clean['Contract End'] = fifa_clean['Contract End'].map(remove_dot_i)

# Assigning the contract start and contract end columns into categories
fifa_clean['Contract Start'] = fifa_clean['Contract Start'].astype('category')
fifa_clean['Contract End'] = fifa_clean['Contract End'].astype('category')

# view samples of the result
fifa_clean[['Contract Type', 'Contract Start', 'Contract End']].loc[397:405]

,Contract Type,Contract Start,Contract End
397,Free,nan,nan
398,Permanent,2019,2023
399,Permanent,2017,2023
400,Permanent,2019,2024
401,Permanent,2014,2026
402,Permanent,2018,2023
403,Permanent,2019,2023
404,Permanent,2019,2024
405,On Loan,2017,2021


#### Cleaning the values in the Club, Name and Full Name columns

The club column values have messy and irrelevant characters in them, this would be cleaned. The Name, Full Name and Club colums have some diacritic characters whic could arise due to different languages and would affect alphabetic ordering during visualizations

In [20]:
# take a look at the column to see the format of naming
fifa_clean['Club'].sample(3)

5695     \n\n\n\nIndependiente Medellín
16135    \n\n\n\nZamora FC             
17617    \n\n\n\nHarrogate Town        
Name: Club, dtype: object

In [21]:
# Replacing the '\n' with an empty string
fifa_clean['Club'] = fifa_clean['Club'].str.replace('\n', '')
fifa_clean['Club'].sample(3)

3518     FC Metz                
14544    SK Brann               
4780     Athletic Club de Bilbao
Name: Club, dtype: object

In [22]:
# Examples of diacritic names
fifa_clean[['Name','Full Name','Club']].loc[110]

Name         N. Süle          
Full Name    Niklas Süle      
Club         FC Bayern München
Name: 110, dtype: object

In [23]:
# Define regular expression pattern to match special characters in Name, LongName and Club columns
def remove_diacritics(text):
    """
    This function removes diacritics (accent marks) from text
    """
    return ''.join(c for c in unicodedata.normalize('NFD', text) if unicodedata.category(c) != 'Mn')

# Apply remove_diacritics function to Name, LongName, and Club columns
fifa_clean['Name'] = fifa_clean['Name'].apply(remove_diacritics)
fifa_clean['Full Name'] = fifa_clean['Full Name'].apply(remove_diacritics)
fifa_clean['Club'] = fifa_clean['Club'].apply(remove_diacritics)

# Check if Diacritic Names were removed
fifa_clean[['Name','Full Name','Club']].loc[110]

Name         N. Sule          
Full Name    Niklas Sule      
Club         FC Bayern Munchen
Name: 110, dtype: object

### 

####  Cleaning the Height column
The height column has values with contrasting units, some are in cm and others in feet, to clean it the units are removed from the values and then they are all converted to ftin in decimals, a descriptive column name would ensure the values are easily understood. The ftin unit is a more appropriate unit here because the weight column is tobe formatted in lbs, with this if need be for creating a BMI column it wont need any more formatting

In [24]:
fifa_clean['Height(ft)'].iloc[845:850]

845    188cm
846    170cm
847    6'3" 
848    6'5" 
849    185cm
Name: Height(ft), dtype: object

In [25]:
# define a function that converts cm to feets and inches, This is my preferred unit as the weight is in lbs, this makes calculations that demands precision like the BMI easy

def convert_cm_to_feet(cm):
    cm = float(cm.replace('cm', '')) # remove 'cm' units and convert to float
    feet = int(cm / 30.48) # calculate number of feet
    inches = round((cm / 2.54) % 12) # calculate number of inches and round to nearest integer
    return f"{feet}'{inches}\"" # return result in feet and inches format

# apply the conversion function only to values that are in cm
fifa_clean['Height(ft)'] = fifa_clean['Height(ft)'].apply(lambda x: convert_cm_to_feet(x) if 'cm' in x else x)

# view the result
fifa_clean['Height(ft)'].iloc[845:850]

845    6'2"
846    5'7"
847    6'3"
848    6'5"
849    6'1"
Name: Height(ft), dtype: object

In [26]:
# define a function to convert feet and inches format to decimal format
def convert_feet_to_dec(feet):
    feet_list = feet.split("'")
    ft = float(feet_list[0])
    if len(feet_list) > 1:
        inches = float(feet_list[1].replace('"', ''))
    else:
        inches = 0
    dec = ft + inches / 12
    return round(dec, 2) # round to 1 decimal place


# apply the conversion function to the 'Height' column
fifa_clean['Height(ft)'] = fifa_clean['Height(ft)'].apply(convert_feet_to_dec)

# view result
fifa_clean['Height(ft)'].iloc[845:850]

845    6.17
846    5.58
847    6.25
848    6.42
849    6.08
Name: Height(ft), dtype: float64

#### Cleaning the Weight Column
The weight column has values with contrasting units, some are in lbs and others in kg, to clean it the units are removed from the values and then they are all converted to lbs, a descriptive column name would ensure the values are easily understood. The lbs unit is a more appropriate unit here because the height column is formatted in ftin, with this if need be for creating a BMI column it wont need any more formatting

In [27]:
fifa_clean['Weight(lbs)'].loc[792:794]

792    75kg  
793    183lbs
794    86kg  
Name: Weight(lbs), dtype: object

In [28]:
# define a function that converts kg to lbs
def convert_kg_to_lbs(kg):
    kg = float(kg.replace('kg', '')) # remove 'kg' units and convert to float
    lbs = int(kg * 2.20462) # calculate kg
    return f"{lbs}lbs" #returns result in lbs format

# apply the conversion function only to values that are in kg
fifa_clean['Weight(lbs)'] = fifa_clean['Weight(lbs)'].apply(lambda x: convert_kg_to_lbs(x) if 'kg' in x else x)

# remove the lbs units from the vale
fifa_clean['Weight(lbs)'] = fifa_clean['Weight(lbs)'].str.replace('lbs', '')

# convert to float datatype
fifa_clean['Weight(lbs)'] = fifa_clean['Weight(lbs)'].astype(float)

# view result of code
fifa_clean['Weight(lbs)'].loc[792:794]

792    165.0
793    183.0
794    189.0
Name: Weight(lbs), dtype: float64

#### Cleaning the Value, Wage and Release Clause columns
The Value, Wage and Release columns have similar mess issues, have currency symbols in them and are formatted in thousand and millions decimal points and have K and M attached to them. These symbols would be dropped and the thousands and millions format would be made uniform. The choice of of the euro(€) currency is because most transfer deals are negotiated in euros

In [29]:
fifa_clean[['Value(€)','Wage(€)', 'Release Clause']].head()

,Value(€),Wage(€),Release Clause
0,€103.5M,€560K,€138.4M
1,€63M,€220K,€75.9M
2,€120M,€125K,€159.4M
3,€129M,€370K,€161M
4,€132M,€270K,€166.5M


In [30]:
# Define a function to convert euro and k/m suffixes to USD and millions
def convert_value(val):
    if val.startswith('€'):
        val = val.replace('€', '')
        if val.endswith('K'):
            val = float(val.replace('K', '')) * 1000
        elif val.endswith('M'):
            val = float(val.replace('M', '')) * 1000000
    
    return pd.to_numeric(val)

# Apply the conversion function to the DataFrame
fifa_clean[['Value(€)', 'Wage(€)', 'Release Clause']] = fifa_clean[['Value(€)', 'Wage(€)', 'Release Clause']].applymap(convert_value)

# Set the display format of floats to include a comma delimiter
pd.options.display.float_format = '{:,.2f}'.format

fifa_clean.rename(columns={'Release Clause': 'Release Clause(€)'}, inplace=True)

# quick glance at the resulting data
fifa_clean[['Value(€)','Wage(€)', 'Release Clause(€)']].sample(5)

,Value(€),Wage(€),Release Clause(€)
12400,"600,000.00","2,000.00","606,000.00"
2054,"4,200,000.00","1,000.00","12,100,000.00"
13248,"475,000.00","1,000.00","473,000.00"
6761,"1,000,000.00","2,000.00","1,000,000.00"
1641,"3,500,000.00","15,000.00","6,000,000.00"


#### Cleaning columns with star ratings (Weaker Foot, Skills, Injury Resistance)
The columns with values as star ratings have symbols to represent the number of stars received for each category by a player, these star symbols would be dropped

In [31]:
fifa_clean[['Weaker Foot(★)','Skils(★)','Injury Resistance(★)']].head()

,Weaker Foot(★),Skils(★),Injury Resistance(★)
0,4 ★,4★,5 ★
1,4 ★,5★,5 ★
2,3 ★,1★,3 ★
3,5 ★,4★,4 ★
4,5 ★,5★,5 ★


In [32]:
fifa_clean['Weaker Foot(★)'] = fifa_clean['Weaker Foot(★)'].str.replace('\u2605', '')
fifa_clean['Skils(★)'] = fifa_clean['Skils(★)'].str.replace('\u2605', '')
fifa_clean['Injury Resistance(★)'] = fifa_clean['Injury Resistance(★)'].str.replace('\u2605', '')

# quick glance at the resulting data
fifa_clean[['Weaker Foot(★)','Skils(★)','Injury Resistance(★)']].sample(5)

,Weaker Foot(★),Skils(★),Injury Resistance(★)
455,3,4,2
16948,2,2,1
4521,4,3,1
6517,3,3,1
14311,4,2,1


#### Current ages of players from the Age column 
The age column houses ages of players in 2021 when this data was collated to have an updated age value for the year of 2023, a new column would be derived from the previous age column

In [33]:
# add 2 to their previous age to get their current age
fifa_clean['Age(2023)'] = fifa_clean['Age(2021)'] + 2

# a quick glance of samples of the data
fifa_clean[['Age(2021)', 'Age(2023)']].sample(5)

,Age(2021),Age(2023)
11405,20,22
9029,25,27
1105,35,37
2966,30,32
3349,24,26


#### Drop unnecessary columns
The contract and loan date end columns have been formatted already to give newer more imformative and cleaner columns(Contract Type, Contract Start and Contract End) so should be drop, the positions column looks messy and a more unique column(Best Positions) is available to tell us what position the player plays at

In [34]:
# drop the contract, positions and loan date end columns
fifa_clean.drop(['Contract', 'Positions', 'Name', 'Loan Date End'], axis=1, inplace=True)
fifa_clean.columns

Index(['ID', 'Full Name', 'photoUrl', 'playerUrl', 'Nationality', 'Age(2021)',
       'Overall Rating', 'Potential Rating', 'Club', 'Height(ft)',
       'Weight(lbs)', 'Preferred Foot', 'Best Overall Rating', 'Best Position',
       'Joined', 'Value(€)', 'Wage(€)', 'Release Clause(€)', 'Attacking',
       'Crossing', 'Finishing', 'Heading Accuracy', 'Short Passing', 'Volleys',
       'Skill', 'Dribbling', 'Curve', 'FK Accuracy', 'Long Passing',
       'Ball Control', 'Movement', 'Acceleration', 'Sprint Speed', 'Agility',
       'Reactions', 'Balance', 'Power', 'Shot Power', 'Jumping', 'Stamina',
       'Strength', 'Long Shots', 'Mentality', 'Aggression', 'Interceptions',
       'Positioning', 'Vision', 'Penalties', 'Composure', 'Defending',
       'Marking', 'Standing Tackle', 'Sliding Tackle', 'Goalkeeping',
       'GK Diving', 'GK Handling', 'GK Kicking', 'GK Positioning',
       'GK Reflexes', 'Total Stats', 'Base Stats', 'Weaker Foot(★)',
       'Skils(★)', 'Attacking Contribution'

#### The total stats column is an aggregate of the Attacking, Skill, Movement, Power, Mentality, Defending and Goal Keeping Column, each of these columns are also aggregates of other columns which are broken down below:
1.  Attacking Column = Crossing + Finishing + HEading Accuracy + Short Passing + Volleys
2. Skill Column = Dribbling + Curve + FK Accuracy + Long Passing + Ball Control 
3. Movement Column = Acceleration + Sprint Speed + Agility + Reactions + Balance 
4. Power Column = Shot Power + Jumping + Stamina + Strength + Long Shots
5. Mentality Column = Aggression + Interceptions + Positioning + Vision + Penalties
6. Defending Column = Marking + Standing Tackle + Sliding Tackle
7. Goalkeeping = GK Diving + GK Handling + GK Kicking + GK Positioning + GK Reflexes
#### This leaves out the composure column, a quick visit to the sofifa.com site shows this is a little oversight and the composure column was supposed to be added to the Mentality Column and the total affects the total stats.

Other attributes column would be accessed to be sure the calculations done were correct and then the composure column values would be inputted where they ought to bee used for calculations

##### Confirm Attacking column Calculations

In [35]:
# Create a new column "Total Attacking" to store the sum of the attacking attributes
fifa_clean["Total Attacking"] = fifa_clean[["Crossing", "Finishing", "Heading Accuracy", "Short Passing", "Volleys"]].sum(axis=1)

# Check if the "Total Attacking" matches the "Attacking" column for each row
fifa_clean["Attacking Check"] = fifa_clean["Attacking"] == fifa_clean["Total Attacking"]

# check to see if there are values that dont tally
fifa_clean[fifa_clean['Attacking Check'] == False]

,ID,Full Name,photoUrl,playerUrl,Nationality,Age(2021),Overall Rating,Potential Rating,Club,Height(ft),Weight(lbs),Preferred Foot,Best Overall Rating,Best Position,Joined,Value(€),Wage(€),Release Clause(€),Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,Weaker Foot(★),Skils(★),Attacking Contribution,Defensive Contribution,Injury Resistance(★),Pace,Shot,Pass Accuracy,Dribbling Ability,Defensive Ability,Physicality,Profile Views,Contract Type,Contract Start,Contract End,Age(2023),Total Attacking,Attacking Check


##### drop columns used in comparison

In [36]:
# Drop the "Total Attacking" and "Attacking Check" columns
fifa_clean = fifa_clean.drop(columns=["Total Attacking", "Attacking Check"])

# a quick glance to confirm thy were dropped
fifa_clean.sample()

,ID,Full Name,photoUrl,playerUrl,Nationality,Age(2021),Overall Rating,Potential Rating,Club,Height(ft),Weight(lbs),Preferred Foot,Best Overall Rating,Best Position,Joined,Value(€),Wage(€),Release Clause(€),Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,Weaker Foot(★),Skils(★),Attacking Contribution,Defensive Contribution,Injury Resistance(★),Pace,Shot,Pass Accuracy,Dribbling Ability,Defensive Ability,Physicality,Profile Views,Contract Type,Contract Start,Contract End,Age(2023)
3273,214874,Marcos Bolados,https://cdn.sofifa.com/players/214/874/21_60.png,http://sofifa.com/player/214874/marcos-bolados/210006/,Chile,24,72,78,Colo-Colo,5.50,158.00,Right,73,CAM,2016-07-26,"3,700,000.00","7,000.00","6,900,000.00",266,67,69,27,70,33,311,72,56,47,65,71,396,88,80,83,63,82,338,70,59,74,67,68,307,55,59,69,68,56,66,175,55,61,59,49,14,9,8,10,8,1842,411,3,3,High,Medium,1,84,67,66,73,55,66,18,Permanent,2016,2025,26


##### Confirm Skill column Calculations

In [37]:
# Create a new column "Total Skill" to store the sum of the skill attributes
fifa_clean["Total Skill"] = fifa_clean[['Dribbling','Curve', 'FK Accuracy', 'Long Passing', 'Ball Control']].sum(axis=1)

# Check if the "Total Skill" matches the "Skill" column for each row
fifa_clean["Skill Check"] = fifa_clean["Skill"] == fifa_clean["Total Skill"]

# check to see if there are values that dont tally
fifa_clean[fifa_clean['Skill Check'] == False]

,ID,Full Name,photoUrl,playerUrl,Nationality,Age(2021),Overall Rating,Potential Rating,Club,Height(ft),Weight(lbs),Preferred Foot,Best Overall Rating,Best Position,Joined,Value(€),Wage(€),Release Clause(€),Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,Weaker Foot(★),Skils(★),Attacking Contribution,Defensive Contribution,Injury Resistance(★),Pace,Shot,Pass Accuracy,Dribbling Ability,Defensive Ability,Physicality,Profile Views,Contract Type,Contract Start,Contract End,Age(2023),Total Skill,Skill Check


##### drop columns used in comparison

In [38]:
# Drop the "Total Skill" and "Skill Check" columns
fifa_clean = fifa_clean.drop(columns=["Total Skill", "Skill Check"])

# a quick glance to confirm they were dropped
fifa_clean.sample()

,ID,Full Name,photoUrl,playerUrl,Nationality,Age(2021),Overall Rating,Potential Rating,Club,Height(ft),Weight(lbs),Preferred Foot,Best Overall Rating,Best Position,Joined,Value(€),Wage(€),Release Clause(€),Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,Weaker Foot(★),Skils(★),Attacking Contribution,Defensive Contribution,Injury Resistance(★),Pace,Shot,Pass Accuracy,Dribbling Ability,Defensive Ability,Physicality,Profile Views,Contract Type,Contract Start,Contract End,Age(2023)
10951,188985,Liridon Kalludra,https://cdn.sofifa.com/players/188/985/21_60.png,http://sofifa.com/player/188985/liridon-kalludra/210006/,Sweden,28,64,64,Kristiansund BK,5.58,149.00,Right,66,CAM,2015-08-06,"575,000.00","1,000.00","563,000.00",276,60,58,44,66,48,308,67,58,53,64,66,395,79,74,90,64,88,325,55,81,74,56,59,297,70,57,62,63,45,63,148,52,51,45,58,8,15,8,11,16,1807,382,3,3,Medium,Medium,1,76,57,63,70,51,65,4,Permanent,2015,2023,30


##### Confirm Movement column Calculations

In [39]:
# Create a new column "Total Movement" to store the sum of the Movement attributes
fifa_clean["Total Movement"] = fifa_clean[['Acceleration', 'Sprint Speed', 'Agility', 'Reactions', 'Balance']].sum(axis=1)

# Check if the "Total Movement" matches the "Movement" column for each row
fifa_clean["Movement Check"] = fifa_clean["Movement"] == fifa_clean["Total Movement"]

# check to see if there are values that dont tally
fifa_clean[fifa_clean['Movement Check'] == False]

,ID,Full Name,photoUrl,playerUrl,Nationality,Age(2021),Overall Rating,Potential Rating,Club,Height(ft),Weight(lbs),Preferred Foot,Best Overall Rating,Best Position,Joined,Value(€),Wage(€),Release Clause(€),Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,Weaker Foot(★),Skils(★),Attacking Contribution,Defensive Contribution,Injury Resistance(★),Pace,Shot,Pass Accuracy,Dribbling Ability,Defensive Ability,Physicality,Profile Views,Contract Type,Contract Start,Contract End,Age(2023),Total Movement,Movement Check


##### drop columns used in comparison

In [40]:
# Drop the "Total Movement" and "Movement Check" columns
fifa_clean = fifa_clean.drop(columns=["Total Movement", "Movement Check"])

# a quick glance to confirm they were dropped
fifa_clean.sample()

,ID,Full Name,photoUrl,playerUrl,Nationality,Age(2021),Overall Rating,Potential Rating,Club,Height(ft),Weight(lbs),Preferred Foot,Best Overall Rating,Best Position,Joined,Value(€),Wage(€),Release Clause(€),Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,Weaker Foot(★),Skils(★),Attacking Contribution,Defensive Contribution,Injury Resistance(★),Pace,Shot,Pass Accuracy,Dribbling Ability,Defensive Ability,Physicality,Profile Views,Contract Type,Contract Start,Contract End,Age(2023)
15278,226728,Lee Desmond,https://cdn.sofifa.com/players/226/728/21_60.png,http://sofifa.com/player/226728/lee-desmond/210006/,Republic of Ireland,25,60,66,St. Patrick's Athletic,6.00,165.00,Left,62,CB,2015-01-01,"350,000.00",500.00,"438,000.00",243,54,37,58,59,35,253,56,57,30,53,57,264,51,48,56,56,53,318,56,72,74,70,46,272,60,57,50,53,52,63,182,61,61,60,53,13,6,15,6,13,1585,332,3,2,High,High,1,49,44,54,56,60,69,2,Permanent,2015,2020,27


##### Confirm Power column Calculations

In [41]:
# Create a new column "Total Power" to store the sum of the Power attributes
fifa_clean["Total Power"] = fifa_clean[['Shot Power', 'Jumping', 'Stamina', 'Strength', 'Long Shots']].sum(axis=1)

# Check if the "Total Power" matches the "Power" column for each row
fifa_clean["Power Check"] = fifa_clean["Power"] == fifa_clean["Total Power"]

# check to see if there are values that dont tally
fifa_clean[fifa_clean["Power Check"] == False]

,ID,Full Name,photoUrl,playerUrl,Nationality,Age(2021),Overall Rating,Potential Rating,Club,Height(ft),Weight(lbs),Preferred Foot,Best Overall Rating,Best Position,Joined,Value(€),Wage(€),Release Clause(€),Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,Weaker Foot(★),Skils(★),Attacking Contribution,Defensive Contribution,Injury Resistance(★),Pace,Shot,Pass Accuracy,Dribbling Ability,Defensive Ability,Physicality,Profile Views,Contract Type,Contract Start,Contract End,Age(2023),Total Power,Power Check


##### drop columns used in comparison

In [42]:
# Drop the "Total Power" and "Power Check" columns
fifa_clean = fifa_clean.drop(columns=["Total Power", "Power Check"])

# a quick glance to confirm they were dropped
fifa_clean.sample()

,ID,Full Name,photoUrl,playerUrl,Nationality,Age(2021),Overall Rating,Potential Rating,Club,Height(ft),Weight(lbs),Preferred Foot,Best Overall Rating,Best Position,Joined,Value(€),Wage(€),Release Clause(€),Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,Weaker Foot(★),Skils(★),Attacking Contribution,Defensive Contribution,Injury Resistance(★),Pace,Shot,Pass Accuracy,Dribbling Ability,Defensive Ability,Physicality,Profile Views,Contract Type,Contract Start,Contract End,Age(2023)
6173,228290,Julien Laporte,https://cdn.sofifa.com/players/228/290/21_60.png,http://sofifa.com/player/228290/julien-laporte/210006/,France,26,69,73,FC Lorient,6.08,180.00,Right,71,CB,2019-07-08,"1,700,000.00","13,000.00","2,500,000.00",208,26,19,70,60,33,204,43,28,25,53,55,279,58,55,49,62,55,316,41,86,81,81,27,235,71,67,29,29,39,59,202,68,69,65,64,14,6,15,15,14,1508,322,3,2,Medium,High,1,56,27,43,49,68,79,4,Permanent,2019,2023,28


##### Update Mentality column by adding Composure values to its attributes

In [43]:
# suming  up the  'Aggression', 'Interceptions', 'Positioning', 'Vision','Penalties'and add the 'Composure' column to make the Mentality column
fifa_clean['Mentality'] = fifa_clean[[ 'Aggression', 'Interceptions', 'Positioning', 'Vision','Penalties', 'Composure']].sum(axis=1)

# a quick glance to confirm they were dropped
fifa_clean[['Mentality', 'Aggression', 'Interceptions', 'Positioning', 'Vision',
       'Penalties', 'Composure']].sample(4)

,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure
5529,379,79,67,52,55,59,67
15784,289,60,54,50,34,42,49
4653,370,63,71,44,67,55,70
13225,281,32,18,63,42,65,61


##### Confirm Defending column Calculations

In [44]:
# Create a new column "Total Defending" to store the sum of the defending attributes
fifa_clean["Total Defending"] = fifa_clean[['Marking', 'Standing Tackle','Sliding Tackle']].sum(axis=1)

# Check if the "Total Defending" matches the "defending" column for each row
fifa_clean["Defending Check"] = fifa_clean["Defending"] == fifa_clean["Total Defending"]

# check to see if there are values that dont tally
fifa_clean[fifa_clean["Defending Check"] == False]

,ID,Full Name,photoUrl,playerUrl,Nationality,Age(2021),Overall Rating,Potential Rating,Club,Height(ft),Weight(lbs),Preferred Foot,Best Overall Rating,Best Position,Joined,Value(€),Wage(€),Release Clause(€),Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,Weaker Foot(★),Skils(★),Attacking Contribution,Defensive Contribution,Injury Resistance(★),Pace,Shot,Pass Accuracy,Dribbling Ability,Defensive Ability,Physicality,Profile Views,Contract Type,Contract Start,Contract End,Age(2023),Total Defending,Defending Check


##### drop columns used in comparison

In [45]:
# Drop the "Total Defending" and "Defending Check" columns
fifa_clean = fifa_clean.drop(columns=["Total Defending", "Defending Check"])

# a quick glance to confirm they were dropped
fifa_clean.sample()

,ID,Full Name,photoUrl,playerUrl,Nationality,Age(2021),Overall Rating,Potential Rating,Club,Height(ft),Weight(lbs),Preferred Foot,Best Overall Rating,Best Position,Joined,Value(€),Wage(€),Release Clause(€),Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,Weaker Foot(★),Skils(★),Attacking Contribution,Defensive Contribution,Injury Resistance(★),Pace,Shot,Pass Accuracy,Dribbling Ability,Defensive Ability,Physicality,Profile Views,Contract Type,Contract Start,Contract End,Age(2023)
10031,232291,Stefan Silva,https://cdn.sofifa.com/players/232/291/21_60.png,http://sofifa.com/player/232291/stefan-silva/210006/,Sweden,30,65,65,AIK,5.92,163.00,Right,67,RW,2018-01-09,"700,000.00","3,000.00","656,000.00",312,62,67,55,63,65,306,68,64,62,46,66,364,77,70,84,64,69,315,59,55,72,65,64,317,33,29,65,67,57,66,94,37,32,25,48,7,13,9,13,6,1690,362,4,3,Medium,Medium,1,73,64,61,69,35,60,2,Permanent,2018,2020,32


##### Confirm Goalkeeping column Calculations

In [46]:
# Create a new column "Total GK" to store the sum of the Goalkeeping attributes
fifa_clean["Total GK"] = fifa_clean[['GK Diving', 'GK Handling','GK Kicking', 'GK Positioning', 'GK Reflexes']].sum(axis=1)

# Check if the "Total Defending" matches the "defending" column for each row
fifa_clean["GK Check"] = fifa_clean["Goalkeeping"] == fifa_clean["Total GK"]

# check to see if there are values that dont tally
fifa_clean[fifa_clean["GK Check"] == False]

,ID,Full Name,photoUrl,playerUrl,Nationality,Age(2021),Overall Rating,Potential Rating,Club,Height(ft),Weight(lbs),Preferred Foot,Best Overall Rating,Best Position,Joined,Value(€),Wage(€),Release Clause(€),Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,Weaker Foot(★),Skils(★),Attacking Contribution,Defensive Contribution,Injury Resistance(★),Pace,Shot,Pass Accuracy,Dribbling Ability,Defensive Ability,Physicality,Profile Views,Contract Type,Contract Start,Contract End,Age(2023),Total GK,GK Check


##### drop columns used in comparison

In [47]:
# Drop the "Total GK" and "GK Check" columns
fifa_clean = fifa_clean.drop(columns=["Total GK", "GK Check"])

# a quick glance to confirm they were dropped
fifa_clean.sample()

,ID,Full Name,photoUrl,playerUrl,Nationality,Age(2021),Overall Rating,Potential Rating,Club,Height(ft),Weight(lbs),Preferred Foot,Best Overall Rating,Best Position,Joined,Value(€),Wage(€),Release Clause(€),Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,Weaker Foot(★),Skils(★),Attacking Contribution,Defensive Contribution,Injury Resistance(★),Pace,Shot,Pass Accuracy,Dribbling Ability,Defensive Ability,Physicality,Profile Views,Contract Type,Contract Start,Contract End,Age(2023)
17889,258581,Daniel Williams,https://cdn.sofifa.com/players/258/581/21_60.png,http://sofifa.com/player/258581/daniel-williams/210006/,Wales,19,54,72,Swansea City,5.58,141.00,Right,56,CM,2019-07-03,"275,000.00","2,000.00","354,000.00",214,41,35,45,64,29,237,49,40,37,57,54,326,70,66,59,49,82,248,47,57,54,41,49,298,56,52,36,53,46,55,156,44,57,55,63,10,15,13,15,10,1487,315,3,2,Medium,Medium,1,68,41,54,53,51,48,1,Permanent,2019,2021,21


#### Total stats represent the final value of all the stats a character or object has, which includes their base stats, any bonuses or modifications gained through equipment or leveling up, and any other factors that affect their abilities. Therefore, adding the base stats to the total stats would result in double-counting the base stats. So base stats would be left out in this aggregation

##### Update Total Stats column by with updating values of its attributes

In [48]:
# suming  up the Attacking, Skill, Movement, Power, Mentality, Defending and Goal Keeping to make the Mentality column
fifa_clean['Total Stats'] = fifa_clean[[ 'Attacking', 'Skill', 'Movement', 'Power','Mentality', 'Defending']].sum(axis=1)

# a view of the resulting total stats column
fifa_clean[[ 'Total Stats','Attacking', 'Skill', 'Movement', 'Power','Mentality', 'Defending']].sample()

,Total Stats,Attacking,Skill,Movement,Power,Mentality,Defending
9793,1582,227,220,299,314,330,192


#### Confirm Base Stats column Calculations
The Base stats refer to the six basic categories of skills used to rate a player's abilities in FIFA. These stats are typically used as a starting point for evaluating a player's performance, and they provide a basic overview of a player's abilities. The six columns would be added to compare if they sum up to the base stats

##### Confirm Base Stats column Calculations

In [49]:
# Create a new column "Total Base Stats" to store the sum of the Base Stats attributes
fifa_clean["Total Base Stats"] = fifa_clean[['Pace', 'Shot',
       'Pass Accuracy', 'Dribbling Ability', 'Defensive Ability',
       'Physicality']].sum(axis=1)

# Check if the "Total Defending" matches the "defending" column for each row
fifa_clean["Base Stats Check"] = fifa_clean["Base Stats"] == fifa_clean["Total Base Stats"]

# check to see if there are values that dont tally
fifa_clean[fifa_clean["Base Stats Check"] == False]

,ID,Full Name,photoUrl,playerUrl,Nationality,Age(2021),Overall Rating,Potential Rating,Club,Height(ft),Weight(lbs),Preferred Foot,Best Overall Rating,Best Position,Joined,Value(€),Wage(€),Release Clause(€),Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,Weaker Foot(★),Skils(★),Attacking Contribution,Defensive Contribution,Injury Resistance(★),Pace,Shot,Pass Accuracy,Dribbling Ability,Defensive Ability,Physicality,Profile Views,Contract Type,Contract Start,Contract End,Age(2023),Total Base Stats,Base Stats Check


##### drop columns used in comparison

In [50]:
# Drop the "Total Base Stats" and "Base Check" columns
fifa_clean = fifa_clean.drop(columns=["Total Base Stats", "Base Stats Check"])

# a view of a sample of the data
fifa_clean.sample()

,ID,Full Name,photoUrl,playerUrl,Nationality,Age(2021),Overall Rating,Potential Rating,Club,Height(ft),Weight(lbs),Preferred Foot,Best Overall Rating,Best Position,Joined,Value(€),Wage(€),Release Clause(€),Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,Weaker Foot(★),Skils(★),Attacking Contribution,Defensive Contribution,Injury Resistance(★),Pace,Shot,Pass Accuracy,Dribbling Ability,Defensive Ability,Physicality,Profile Views,Contract Type,Contract Start,Contract End,Age(2023)
12114,251440,Lukas Scepanik,https://cdn.sofifa.com/players/251/440/21_60.png,http://sofifa.com/player/251440/lukas-scepanik/210006/,Germany,26,63,64,MSV Duisburg,5.75,154.00,Left,63,LM,2019-07-01,"575,000.00",850.00,"648,000.00",281,62,60,39,64,56,289,64,58,52,54,61,346,74,69,74,50,79,305,59,67,65,57,57,329,58,48,66,58,52,47,145,45,50,50,49,11,10,10,7,11,1695,361,4,2,High,Medium,1,71,59,60,64,47,60,3,Permanent,2019,2021,28


#### Reindex Columns

Changing the order of columns can be helpful for organizing the data in a way that makes it easier to understand or analyze.

In [51]:
fifa_clean.columns

Index(['ID', 'Full Name', 'photoUrl', 'playerUrl', 'Nationality', 'Age(2021)',
       'Overall Rating', 'Potential Rating', 'Club', 'Height(ft)',
       'Weight(lbs)', 'Preferred Foot', 'Best Overall Rating', 'Best Position',
       'Joined', 'Value(€)', 'Wage(€)', 'Release Clause(€)', 'Attacking',
       'Crossing', 'Finishing', 'Heading Accuracy', 'Short Passing', 'Volleys',
       'Skill', 'Dribbling', 'Curve', 'FK Accuracy', 'Long Passing',
       'Ball Control', 'Movement', 'Acceleration', 'Sprint Speed', 'Agility',
       'Reactions', 'Balance', 'Power', 'Shot Power', 'Jumping', 'Stamina',
       'Strength', 'Long Shots', 'Mentality', 'Aggression', 'Interceptions',
       'Positioning', 'Vision', 'Penalties', 'Composure', 'Defending',
       'Marking', 'Standing Tackle', 'Sliding Tackle', 'Goalkeeping',
       'GK Diving', 'GK Handling', 'GK Kicking', 'GK Positioning',
       'GK Reflexes', 'Total Stats', 'Base Stats', 'Weaker Foot(★)',
       'Skils(★)', 'Attacking Contribution'

In [52]:
fifa_clean.insert(loc=7, column='Age(2023)', value=fifa_clean.pop('Age(2023)'))
fifa_clean.insert(loc=8, column='Contract Type', value=fifa_clean.pop('Contract Type'))
fifa_clean.insert(loc=9, column='Contract Start', value=fifa_clean.pop('Contract Start'))
fifa_clean.insert(loc=10, column='Contract End', value=fifa_clean.pop('Contract End'))
fifa_clean.insert(loc=66, column='Pace', value=fifa_clean.pop('Pace'))
fifa_clean.insert(loc=67, column='Shot', value=fifa_clean.pop('Shot'))
fifa_clean.insert(loc=68, column='Pass Accuracy', value=fifa_clean.pop('Pass Accuracy'))
fifa_clean.insert(loc=69, column='Dribbling Ability', value=fifa_clean.pop('Dribbling Ability'))
fifa_clean.insert(loc=70, column='Defensive Ability', value=fifa_clean.pop('Defensive Ability'))
fifa_clean.insert(loc=71, column='Physicality', value=fifa_clean.pop('Physicality'))


fifa_clean.columns

Index(['ID', 'Full Name', 'photoUrl', 'playerUrl', 'Nationality', 'Age(2021)',
       'Overall Rating', 'Age(2023)', 'Contract Type', 'Contract Start',
       'Contract End', 'Potential Rating', 'Club', 'Height(ft)', 'Weight(lbs)',
       'Preferred Foot', 'Best Overall Rating', 'Best Position', 'Joined',
       'Value(€)', 'Wage(€)', 'Release Clause(€)', 'Attacking', 'Crossing',
       'Finishing', 'Heading Accuracy', 'Short Passing', 'Volleys', 'Skill',
       'Dribbling', 'Curve', 'FK Accuracy', 'Long Passing', 'Ball Control',
       'Movement', 'Acceleration', 'Sprint Speed', 'Agility', 'Reactions',
       'Balance', 'Power', 'Shot Power', 'Jumping', 'Stamina', 'Strength',
       'Long Shots', 'Mentality', 'Aggression', 'Interceptions', 'Positioning',
       'Vision', 'Penalties', 'Composure', 'Defending', 'Marking',
       'Standing Tackle', 'Sliding Tackle', 'Goalkeeping', 'GK Diving',
       'GK Handling', 'GK Kicking', 'GK Positioning', 'GK Reflexes',
       'Total Stats', 'Bas

In [53]:
# save cleaned fifa data
fifa_clean.to_csv('Fifa_2021_cleaned.csv',index=False)

# Print a message to confirm the file has been saved
print('fifa_clean.csv saved successfully.')

fifa_clean.csv saved successfully.


# References
### https://sofifa.com/